<a href="https://colab.research.google.com/github/merucode/DL/blob/92-Colab-Kaggle-ML-Classification/11-01_%5B%5D_Detect-sleep-states(basic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imformation

* Title : [Detect Sleep States](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/data)
* Type : Multiclass Classification
* Evaluation : -
* Model : -
* Python version: 3.10.12
* Basic library version
  * sklearn(scikit-learn==1.2.2)
  * numpy(numpy==1.23.5)
  * pandas(pandas==1.5.3)
  * matplotlib(matplotlib==3.7.1)
  * seaborn(seaborn==0.12.2)
* Addtional Library version
* Considering Library version
* Improvement

# STEP 0. Version check and Install Dependency

Step 0-1. Install Dependency

Step 0-2. Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.version}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

In [ ]:
!pip list

Step 0-3. Download Data

In [ ]:
!export KAGGLE_USERNAME=*** && export KAGGLE_KEY=*** && kaggle competitions download -c child-mind-institute-detect-sleep-states

In [ ]:
from zipfile import ZipFile

data_path = '/content/'

with ZipFile(data_path + 'child-mind-institute-detect-sleep-states.zip') as zipper:
  zipper.extractall()

# STEP 1. Check Data


### Data 정보 및 분석 결과

* train_series.parquet
  * series_id- 각 가속도계 시리즈의 고유 식별자입니다.
  * step- 계열 내의 각 관측값에 대한 정수 시간 간격입니다.
  * timestamp- ISO 8601 형식의 해당 날짜/시간 %Y-%m-%dT%H:%M:%S%z.
  * anglez- GGIR 패키지 에서 계산 및 설명된 바와 같이 , z 각도는 수면 감지에 일반적으로 사용되는 개별 가속도계 구성 요소에서 파생된 측정 기준으로, 신체의 수직 축에 대한 팔의 각도를 나타냅니다.
  * enmo- GGIR 패키지 에 의해 계산되고 설명된 대로 ENMO는 모든 가속도계 신호 중 유클리드 표준 마이너스 하나이며 음수 값은 0으로 반올림됩니다. 이 공간에는 가속도에 대한 표준 측정값이 없지만 이는 일반적으로 계산되는 여러 기능 중 하나입니다.
* test_series.parquet - 위와 동일한 필드를 포함하는 테스트 데이터
* train_events.csv - 시작 및 깨우기 이벤트를 기록하는 훈련 세트의 시리즈에 대한 수면 로그입니다.
  * series_id- `train_series.parquet`  각 가속도계 데이터 계열에 대한 고유 식별자입니다.
  * nightonset- `onset / wakeup` 사건 쌍 의 열거 . 매일 밤 최대 한 쌍의 이벤트가 발생할 수 있습니다.
  * event- 이벤트 유형(`onset or wakeup`)입니다.
  * step- timestamp가속도계 시리즈에 기록된 이벤트 발생 시간

### Code

Step 1-1. Check data

In [ ]:
import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/content/'

train_events = pd.read_csv(data_path + 'train_events.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
train_events.shape, submission.shape

In [ ]:
train_events

In [ ]:
submission.head(2)

In [ ]:
train_events.info()

In [ ]:
print(train_events.groupby('series_id').head(1)["event"].unique())  # Check that all of the series start off in the awake state (i.e. the very first event is the onset of sleep):

print(train_events.groupby('series_id').tail(1)["event"].unique())  # Check that all of the series end in the awake state (i.e. the very last event is a wakeup):

series_has_NaN = train_events.groupby('series_id')['step'].apply(lambda x: x.isnull().any())  # How many series have NaN values?
print(series_has_NaN.value_counts())

no_NaN_series = series_has_NaN[~series_has_NaN].index.tolist()  # We see the following 37 series have no NaN values whatsoever:

# also drop these two "truncated" events series seen in EDA:
no_NaN_series.remove('31011ade7c0a') # incomplete events data
no_NaN_series.remove('a596ad0b82aa') # incomplete events data

no_NaN_series

In [ ]:
def get_train_series(series):
    train_series = pd.read_parquet(data_path + "train_series.parquet", filters=[('series_id','=',series)])
    train_events = pd.read_csv(data_path + "train_events.csv").query('series_id == @series')

    train_events = train_events.dropna()
    train_events["step"]  = train_events["step"].astype("int")
    train_events["awake"] = train_events["event"].replace({"onset":1,"wakeup":0})

    train = pd.merge(train_series, train_events[['step','awake']], on='step', how='left')
    train["awake"] = train["awake"].bfill(axis ='rows')
    # final section:
    # train_events.groupby('series_id').tail(1)["event"].unique()
    # Result: the last event is always a "wakeup"
    train['awake'] = train['awake'].fillna(1) # awake
    train["awake"] = train["awake"].astype("int")
    return(train)

In [ ]:
import gc
import matplotlib.pyplot as plt
import seaborn as sns

smaller_train_data = []

for series_id in no_NaN_series:
    train = get_train_series(series_id)
    smaller_train_data.append(train)

    # vizualize these series
    #display(Markdown('###  anglez for series ' + series_id))
    fig, ax = plt.subplots(figsize=(20, 3))
    sns.lineplot(data=train, x="step", y="anglez",hue="awake", linewidth = 0.5)
    plt.show();
    #display(Markdown('###  enmo for series ' + series_id))
    fig, ax = plt.subplots(figsize=(20, 3))
    sns.lineplot(data=train, x="step", y="enmo",hue="awake", linewidth = 0.5)
    plt.show();
    del train
    gc.collect();

*** https://www.kaggle.com/code/carlmcbrideellis/zzzs-make-small-starter-datasets-target


In [ ]:
### 결측값(-1) 확인
import numpy as np
import missingno as msno

# 훈련 데이터 복사본에서 -1을 np.NaN로 변환
train_copy = train.copy().replace(-1, np.NaN)

# 결측값 시각화(처음 28개만)
msno.bar(df=train_copy.iloc[:, 1:28], figsize=(13, 6))  # ★ ps_reg_03, ps_car_03_cat, ps_car_05_cat 결측값 많음

In [ ]:
# 나머지 결측값 시각화
msno.bar(df=train_copy.iloc[:, 28:], figsize=(13, 6)) # ★ ps_car_14 결측값 존재

In [ ]:
# 결측값 메트릭스
msno.matrix(df=train_copy.iloc[:, 1:28], figsize=(13, 6))

In [ ]:
### 피쳐 요약표
def resumetable(df):
  print(f"데이터셋 형상: {df.shape}")
  summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
  #summary = summary.reset_index()
  #summary = summary.rename(columns={'index': '피쳐'})
  summary['결측값 갯수'] = (df == -1).sum().values # 피처별 -1 개수(이번 데이터 특성)
  summary['고유값 개수'] = df.nunique().values
  summary['데이터 종류'] = None
  for col in df.columns:
    if 'bin' in col or col =='target':
      summary.loc[col, '데이터 종류'] = '이진형'
    elif 'cat' in col:
      summary.loc[col, '데이터 종류'] = '명목형'
    elif df[col].dtype == float:
      summary.loc[col, '데이터 종류'] = '연속형'
    elif df[col].dtype ==int:
      summary.loc[col, '데이터 종류'] = '순서형'

  return summary

summary = resumetable(train)
summary

In [ ]:
### 데이터 요약표를 통한 칼럼 호출
summary[summary['데이터 타입'] == 'float64'].index

Step 1-2. Features Info


★ 분석결과
* ps_reg_03, ps_car_03_cat, ps_car_05_cat, ps_car_14 결측값 많음

Step 1-3. Feature Engineering

# STEP 2. Data Visualize
★ 분석결과
* 타깃 피쳐
  * 타깃값의 비율차이가 큼 → 작은 타깃값 잘 예측하는 게 중요 → 고윳값 별 타깃값 비율 확인 필요
  * 고윳값 마다 타깃값 비율이 다르고, 신뢰구간이 좁아야 유효한 피쳐
* 이진 피쳐
  * ps_ind_10_bin ~ ps_ind_13_bin : 신뢰구간이 넓어서 통계적 유효성이 떨어짐 → 제거
  * ps_calc_15_bin ~ ps_calc_20_bin : 고윳값 0, 1에 대한 타깃값 차이가 없음 → 타깃값 예측력이 없으므로 제거
* 명목 피쳐
  * 결측값에 대한 타깃값 비율도 신뢰구간이 넓다는 점을 감안해도 다른 값과 차이 존재 → 결측값 예측력 존재 하므로 모두 이용
* 순서 피쳐
  * ps_ind_14 : 타깃값 비율의 신뢰구간이 넓어 통계적 유효성이 떨어짐 → 삭제
  * ps_calc_04 ~ ps_cal_14 : 타깃값 비율 차이 없거나 신뢰구간 넓음 → 삭제
* 연속 피쳐
  * ps_calc_01 ~ ps_calc_03 : 타깃값 비율 차이 없음 → 제거
  * (상관관계) ps_car_12, ps_car_14 강한 상관관계 : ps_car_14 제거

STEP 2-1. Count Plot

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### 네모 도형 객체를 순회화며 막대 상단에 타깃값 비율 표시
def write_percent(ax, total_size):
  for patch in ax.patches:
    height = patch.get_height() # 도형 높이(데이터 개수)
    width = patch.get_width()   # 도형 너비
    left_coord = patch.get_x()  # 도형 왼쪽 테두리의 x축 위치
    percent = height/total_size*100

    # x,y 좌표에 텍스트 입력
    ax.text(x=left_coord + width/2.0,
            y=height + total_size*0.001,
            s=f'{percent:1.1f}%',
            ha='center')

In [ ]:
### Count Plot : 범주형 데이터 갯수 확인
mpl.rc('font', size=15) # 폰트 크기를 15로 설정
plt.figure(figsize=(3, 3))

# 타깃값 분포 카운트플롯
ax = sns.countplot(x='target', data=train)
write_percent(ax, len(train)) # 비율 표시
ax.set_title('Target Distributuion')
# ★ 타깃값의 비율차이가 큼 → 작은 타깃값 잘 예측하는 게 중요 → 고윳값 별 타깃값 비율 확인 필요
# ★ 고윳값 마다 타깃값 비율이 다르고, 신뢰구간이 좁아야 유효한 피쳐

STEP 2-2. Binary Feature

In [ ]:
import matplotlib.gridspec as gridspec

### 이진 피처 Bar Plot에 타깃 비율 같이 그리기
def plot_target_ratio_by_features(df, features, num_rows, num_cols, size=(12,18)):
  mpl.rc('font', size=9)
  plt.figure(figsize=size)
  grid = gridspec.GridSpec(num_rows, num_cols)  # 서브플롯 배치
  plt.subplots_adjust(wspace=0.3, hspace=0.3)  # 서브플롯 여백

  for idx, feature in enumerate(features):
    ax = plt.subplot(grid[idx])
    # ax축에 고윳값별 타깃값 1 비율 막대 그래프 그리기
    sns.barplot(x=feature, y='target', data=df, palette='Set2', ax=ax)

In [ ]:
bin_features = summary[summary['데이터 종류']=='이진형'].index # 이진 피처
plot_target_ratio_by_features(train, bin_features, num_rows=6, num_cols=3)

# ★ ps_ind_10_bin ~ ps_ind_13_bin : 신뢰구간이 넓어서 통계적 유효성이 떨어짐 → 제거
# ★ ps_calc_15_bin ~ ps_calc_20_bin : 고윳값 0, 1에 대한 타깃값 차이가 없음 → 타깃값 예측력이 없으므로 제거

Step 2-3 Nominal Feature

In [ ]:
nom_features = summary[summary['데이터 종류']=='명목형'].index
plot_target_ratio_by_features(train, nom_features, num_rows=7, num_cols=2)

# ★ ps_ind_02_cat : 결측값에 대한 타깃값 비율 많음 → 결측값 자체가 타깃값에 대한 예측력 존재
# ★ ps_car_02_cat : 결측값에 대한 타깃값 비율 0 → 결측값 자체가 타깃값에 대한 예측력 존재
# ★ 그 외 결측값에 대한 타깃값 비율도 신뢰구간이 넓다는 점을 감안해도 다른 값과 차이 존재 → 결측값 예측력 존재 하므로 모두 이용

Step 2-4. Orderial Feature

In [ ]:
ord_features = summary[summary['데이터 종류']=='순서형'].index
plot_target_ratio_by_features(train, ord_features, num_rows=8, num_cols=2)

# ★ ps_ind_14 : 타깃값 비율의 신뢰구간이 넓어 통계적 유효성이 떨어짐 → 삭제
# ★ ps_calc_04 ~ ps_cal_14 : 타깃값 비율 차이 없거나 신뢰구간 넓음 → 삭제

Step 2-5. Continuous Feature

In [ ]:
# cut 함수로 연속 구간을 나눈뒤 출력
cont_features = summary[summary['데이터 종류']=='연속형'].index

plt.figure(figsize=(12, 16))
grid = gridspec.GridSpec(5, 2)  # 서브플롯 배치
plt.subplots_adjust(wspace=0.3, hspace=0.3)  # 서브플롯 여백

for idx, cont_feature in enumerate(cont_features):
  # 값을 5개 구간으로 나누기
  train[cont_feature] = pd.cut(train[cont_feature], 5)

  ax = plt.subplot(grid[idx])
  sns.barplot(x=cont_feature, y='target', data=train, palette='Set2', ax=ax)
  ax.tick_params(axis='x', labelrotation=10)

# ★ ps_calc_01 ~ ps_calc_03 : 타깃값 비율 차이 없음 → 제거

Step 2-6. Continuous Feature(Correlation)

In [ ]:
# np.NaN 값 삭제
train_copy = train_copy.dropna()

In [ ]:
plt.figure(figsize=(10,8))
cont_corr = train_copy[cont_features].corr()    # 연속형 피쳐 상관관계
sns.heatmap(cont_corr, annot=True, cmap='OrRd') # 히트맵 그리기

# ★ ps_car_12, ps_car_14 강한 상관관계 : ps_car_14 제거 후 성능 상승
# ★ ps_reg_02, ps_reg_03 강한 상관관계 : ps_reg_03 삭제 후 성능 저하로 유지

# STEP 3. Feature Engineering

Step 3-1. Load Data

In [ ]:
import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/content/'

# 훈련, 검증, 테스트 데이터 경로 설정
train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

Step 3-2. Concat Data(Apply same feature engineering with train, test)

In [ ]:
all_data = pd.concat([train, test], ignore_index=True) # 훈련 데이터와 테스트 데이터 합치기
all_data = all_data.drop('target', axis=1)  # 타깃값 제거
all_data.tail(3)

In [ ]:
all_features = all_data.columns # 전체 피처
all_features

Step 3-3. Norminal: One-Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat'in feature]

onehot_encoder = OneHotEncoder()   # 원핫인코더 생성
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

Step 3-4. Remove Feature

In [ ]:
# 불필요한 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# 1) 명목형 피쳐, 2) calc 분류의 피처, 3) 추가 제거할 피처를 제외한 피처
remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and
                          'calc' not in feature and
                          feature not in drop_features)]

Step 3-5. Merge Feature

In [ ]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

Step 3-6. Divide Data(train, test, valid)

In [ ]:
num_train = len(train)  # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

# STEP 4. Evaluation Function

In [ ]:
### 정규화된 지니계수
import numpy as np

def eval_gini(y_true, y_pred):
  # 실제값과 예측값의 크기가 같은지 서로 확인(다르면 오류)
  assert y_true.shape == y_pred.shape

  n_samples = y_true.shape[0]                       # 데이터 개수
  L_mid = np.linspace(1 / n_samples, 1, n_samples)  # 대각선 값

  # 1) 예측값에 대한 지니계수
  pred_order = y_true[y_pred.argsort()] # y_pred 크기 순으로 y_true 값 정렬
  L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
  G_pred = np.sum(L_mid - L_pred) # 예측값에 대한 지니 계수

  # 2) 예측이 완벽할 때 지니계수
  true_order = y_true[y_true.argsort()] # y_true 크기 순으로 y_true 값 정렬
  L_true = np.cumsum(true_order) / np.sum(true_order)  # 로렌츠 곡선
  G_true = np.sum(L_mid - L_true) # 예측값이 완벽할 때 지니계수

  # 정규화된 지니계수
  return G_pred / G_true

In [ ]:
# LightGBM용 gini() 함수
def gini(preds, dtrain):
  labels = dtrain.get_label()
  return 'gini', eval_gini(labels, preds), True   # 평가지표 이름, 평가점수, 평가 점수가 높을수록 좋은지 여부

# STEP 4. Model


In [ ]:
import lightgbm as lgb

# STEP 5. Learning

Step 5-1. Setting

In [ ]:
from sklearn.model_selection import StratifiedKFold

# K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# LightGBM 하이퍼파라미터 설정
params = {'objective':'binary',
          'learning_rate':0.01,
          'force_row_wise':True,  # 경고 문구 제거
          'random_state':0}

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

Step 5-2. Learning(OOF)

In [ ]:
# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
  # 각 폴드를 구분하는 문구 출력
  print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)

  # 훈련용 데이터, 검증용 데이터 설정
  X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
  X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

  # LightGBM 전용 데이터셋 생성
  dtrain = lgb.Dataset(X_train, y_train)
  dvalid = lgb.Dataset(X_valid, y_valid)

  # LightGBM 모델 훈련
  lgb_model = lgb.train(params=params,          # 훈련용 하이퍼파라미터
                        train_set=dtrain,       # 훈련 데이터셋
                        num_boost_round=1000,   # 부스팅 반복 횟수
                        valid_sets=dvalid,      # 검증 데이터셋
                        feval=gini,             # 평가지표
                        early_stopping_rounds=100,  # 조기종료 조건
                        verbose_eval=100)       # 100번째마다 점수 출력

  # 테스트 데이터를 활용해 OOF 예측
  oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
  # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
  oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

  # 검증 데이터 예측 확률에 대한 정규화 지니계수
  gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
  print(f"폴드 {idx+1} 지니계수: {gini_score}\n")

# STEP 6. Prediction and Submission

Step 6-1. Prediction

In [ ]:
print('OOF 검증 데이터 지니계수: ', eval_gini(y, oof_val_preds))

Step 6-2. Submission

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')   # 제출 파일 생성